In [7]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
import yaml

from pyspark.sql import SparkSession

plt.style.use("seaborn-white")

In [8]:
conf_dict = yaml.safe_load(Path("../config/conf.yaml").read_text())

In [9]:
conf_dict

{'ownership_data_input': 'data/raw/statements.latest.jsonl',
 'ownership_data_output': 'data/interim/gb_ownership.parquet'}

In [13]:
ownership_data_output_abs = str(
    Path("..", conf_dict["ownership_data_output"]).absolute()
)

In [2]:
spark = SparkSession.builder.getOrCreate()

22/05/24 22:57:54 WARN Utils: Your hostname, domvwt-XPS-13-9305 resolves to a loopback address: 127.0.1.1; using 192.168.0.24 instead (on interface wlp164s0)
22/05/24 22:57:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/24 22:57:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
sdf00 = spark.read.parquet(f"file://{ownership_data_output_abs}")

In [18]:
data_desc = sdf00.describe().toPandas()

,summary,birthDate,dissolutionDate,entityType,foundingDate,missingInfoReason,name,personType,statementDate,statementID,statementType
0,count,0,1849577,7246475,7194217,0,7203721,0,0,7246475,7246475
1,mean,None,None,None,None,None,3969408.4274999998,None,None,None,None
2,stddev,None,None,None,None,None,7914167.174474322,None,None,None,None
3,min,None,1991-04-05,registeredEntity,1856-09-25,None,! LTD,None,None,openownership-register-10000003642855225556,entityStatement
4,max,None,2022-04-19,registeredEntity,2022-04-12,None,€URR£N€¥ LIFESTYLE LIMITED,None,None,openownership-register-9999996837221681050,entityStatement


In [23]:
sample_sdf = sdf00.limit(100)

In [24]:
sample_sdf.printSchema()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- birthDate: string (nullable = true)
 |-- dissolutionDate: string (nullable = true)
 |-- entityType: string (nullable = true)
 |-- foundingDate: string (nullable = true)
 |-- identifiers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- scheme: string (nullable = true)
 |    |    |-- schemeName: string (nullable = true)
 |    |    |-- uri: string (nullable = true)
 |-- incorporatedInJurisdiction: struct (nullable = true)
 |    |-- code: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- interestedParty: struct (nullable = true)
 |    |-- describedByEntityStatement: string (nullable = true)
 |    |-- describedByPersonStatement: string (nullable = tru

In [31]:
sample_sdf.limit(10).toPandas()

,addresses,birthDate,dissolutionDate,entityType,foundingDate,identifiers,incorporatedInJurisdiction,interestedParty,interests,missingInfoReason,name,names,nationalities,personType,source,statementDate,statementID,statementType,subject
0,"[(106 Birmingham Road, Bromsgrove, Worcestersh...",None,None,registeredEntity,1997-03-11,"[(03331247, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,GARDNERS MEADOW LIMITED,None,None,None,None,None,openownership-register-4525703773586944376,entityStatement,None
1,"[(Eden Court, Crow Hill Drive, Mansfield, Nott...",None,None,registeredEntity,2007-03-02,"[(OC326488, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,HOPKINS SOLICITORS LLP,None,None,None,None,None,openownership-register-6055736579175054779,entityStatement,None
2,"[(The Estate Office, Church Mews, Whippingham,...",None,None,registeredEntity,1990-02-26,"[(02474472, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,FIRBANK LODGE (SHANKLIN) MANAGEMENT COMPANY LI...,None,None,None,None,None,openownership-register-18127841203727953625,entityStatement,None
3,"[(48 Bryant Road, Kettering, NN15 6JG, GB, reg...",None,2017-12-19,registeredEntity,2016-03-03,"[(10041142, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,AIDE MEDICAL LIMITED,None,None,None,None,None,openownership-register-12010780370140288703,entityStatement,None
4,"[(13-15 Creek Road, East Molesey, KT8 9BE, GB,...",None,None,registeredEntity,2012-12-24,"[(08340445, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,ZERO BILLS HOME LIMITED,None,None,None,None,None,openownership-register-3546719582361560732,entityStatement,None
5,"[(Millbank Tower 1st Floor, 21-24 Millbank, Lo...",None,2019-11-05,registeredEntity,2013-02-20,"[(08411380, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,FOR BRITAIN LIMITED,None,None,None,None,None,openownership-register-208692845369555487,entityStatement,None
6,"[(Brunel House 340 Firecrest Court, Centre Par...",None,2019-01-15,registeredEntity,2016-03-03,"[(10039544, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,JR BARRINGER CONSULTING LIMITED,None,None,None,None,None,openownership-register-5354712901499440762,entityStatement,None
7,"[(2 Hills Road, Cambridge, Cambridgeshire, CB2...",None,None,registeredEntity,2014-02-19,"[(08901891, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,SWAN PLACE (CIPPENHAM) MANAGEMENT LIMITED,None,None,None,None,None,openownership-register-3235557873294634078,entityStatement,None
8,"[(33 Hoghton Street, Southport, Merseyside, PR...",None,None,registeredEntity,2011-03-03,"[(07550790, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,NELSON COURT (SOUTHPORT) RTM COMPANY LIMITED,None,None,None,None,None,openownership-register-5134206245602809181,entityStatement,None
9,"[(66 Harley Street, Stoke-On-Trent, ST1 3LA, G...",None,2019-04-23,registeredEntity,2017-03-03,"[(10649847, GB-COH, Companies House, None), (h...","(GB, United Kingdom of Great Britain and North...",None,None,None,WISHING TREE SOT LIMITED,None,None,None,None,None,openownership-register-9179865058192705681,entityStatement,None


In [40]:
sdf00.filter(~F.isnull("names")).show()

+---------+---------+---------------+----------+------------+-----------+--------------------------+---------------+---------+-----------------+----+-----+-------------+----------+------+-------------+-----------+-------------+-------+
|addresses|birthDate|dissolutionDate|entityType|foundingDate|identifiers|incorporatedInJurisdiction|interestedParty|interests|missingInfoReason|name|names|nationalities|personType|source|statementDate|statementID|statementType|subject|
+---------+---------+---------------+----------+------------+-----------+--------------------------+---------------+---------+-----------------+----+-----+-------------+----------+------+-------------+-----------+-------------+-------+
+---------+---------+---------------+----------+------------+-----------+--------------------------+---------------+---------+-----------------+----+-----+-------------+----------+------+-------------+-----------+-------------+-------+

